<a href="https://colab.research.google.com/github/karimghabbour/VirtualBuddy_RAG/blob/main/HPE_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Step 1: Download GPT4ALL Model

In [2]:
#Model in drive
#!wget https://gpt4all.io/models/ggml-gpt4all-j-v1.3-groovy.bin

#Step 2:Download Documents to be Accessed

In [3]:
#Mount Google Drive For Access
from google.colab import drive
drive.mount('/content/drive')

# Define the base path to the folder containing your PDFs
base_path = '/content/drive/My Drive/BasicRAG/'

# List of your PDF files
pdf_files = [
    'Resources inside HPE Barcelona Hub.pdf',
    'uncover-hidden-insights-create-business-advantage.pdf',
    'HPE GreenLake FAQ for technology innovators frequently asked questions-a00107144enw.pdf',
    'HPE Tech Care Service-a00108652enw.pdf'
]

# Full paths to the PDF files
pdf_paths = [base_path + pdf for pdf in pdf_files]



Mounted at /content/drive


# Download Extra Packages

In [4]:
!pip install pymupdf pygpt4all sentence_transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 14.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.0/362.0 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.0/246.0 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 

# Download Llama Index Package

In [5]:
!pip install langchain
!pip install llama-index
%pip install llama-index-embeddings-langchain
%pip install llama-index-llms-langchain



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.8/276.8 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 14.9 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 79.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

# Document Set up

In [6]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.llms import GPT4All
from llama_index.core.node_parser import SimpleFileNodeParser
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.core.indices.vector_store.base import VectorStoreIndex
from llama_index.embeddings.langchain import LangchainEmbedding
from llama_index.llms.langchain import LangChainLLM #instead of LLMPredictor
from llama_index.core import ServiceContext
from llama_index.core import StorageContext
from llama_index.core import download_loader
from llama_index.core import PromptHelper



In [7]:
PyMuPDFReader = download_loader("PyMuPDFReader")


<ipython-input-7-e65a49cf0750>:1: DeprecationWarning: Call to deprecated function (or staticmethod) download_loader. (`download_loader()` is deprecated. Please install tool using pip install directly instead.)
  PyMuPDFReader = download_loader("PyMuPDFReader")


In [8]:
# Load and prepare all documents
#documents = []
#for pdf_path in pdf_paths:
    # Load the document using PyMuPDFReader
    #loaded_docs = PyMuPDFReader().load(file_path=pdf_path, metadata=True)

    # No need to decode if the text is already a string
    #for doc in loaded_docs:
    #    documents.append(doc)



In [24]:
# Load and prepare all documents
documents = []
for pdf_path in pdf_paths:
    # Load the document using PyMuPDFReader
    loaded_docs = PyMuPDFReader().load(file_path=pdf_path, metadata=True)

    # Add file extension to the document's metadata
    for doc in loaded_docs:
        extension = pdf_path.split('.')[-1]  # Get the file extension
        if not hasattr(doc, 'metadata'):
            doc.metadata = {}  # Create metadata attribute if it doesn't exist
        doc.metadata['extension'] = extension  # Set the 'extension' in the document's metadata
        documents.append(doc)


In [25]:
#testing docs
for i, pdf_path in enumerate(pdf_paths):
    print(f"First page text from: {pdf_path.split('/')[-1]}")  # Prints the file name
    print(documents[i].text[:500])  # Prints the first 500 characters to avoid too much output
    print("\n---\n")


First page text from: Resources inside HPE Barcelona Hub.pdf
Resources inside HPE Barcelona Hub:  
 
- 
Issues with Laptops: Reach out to Karim Ghabbour 
- 
Issues with Wifi: Reach out to Johaness Hellwig 
- 
Info about NGQ and quoting: Reach out to Karim Hellwig  


---

First page text from: uncover-hidden-insights-create-business-advantage.pdf
Uncover hidden insights to 
create a business advantage 
Three stories reveal how you can unlock new 
opportunities, gain a competitive edge, and 
maximize the potential for success.


---

First page text from: HPE GreenLake FAQ for technology innovators frequently asked questions-a00107144enw.pdf
Finding new ways to do business is always a goal of future-driven 
companies. When you look at your organization’s data, you want to 
view it through a fresh set of eyes. A new perspective avoids the 
same old approach to data analytics, regardless of how well it has 
served your organization in the past. 
Taking a different view of your data may un

In [27]:
!pip install gpt4all

In [28]:
# Initialize the GPT4All Model
local_llm_path = '/content/drive/My Drive/BasicRAG/gpt4all-falcon-newbpe-q4_0.gguf'
llm = GPT4All(model=local_llm_path, backend='gptj', streaming=True)

# Initialize the LLMPredictor
llm_predictor = LangChainLLM(llm=llm)

# Initialize the embedding model
embed_model = LangchainEmbedding(HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))


#Initialize RAG Components

In [29]:
prompt_helper = PromptHelper(chunk_size_limit=512, num_output=256, chunk_overlap_ratio=0.1)
service_context = ServiceContext.from_defaults(
    llm_predictor=llm_predictor,
    embed_model=embed_model,
    prompt_helper=prompt_helper,
    node_parser=SimpleFileNodeParser(text_splitter=TokenTextSplitter(chunk_size=300, chunk_overlap=20))
)

LLMPredictor is deprecated, please use LLM instead.


<ipython-input-29-767d510879be>:2: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


#Creating Index

In [30]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

AttributeError: 'tuple' object has no attribute 'ref_doc_id'